# 0. Imports

In [0]:
# Imports
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])

# 1. Introduction

This notebooks aims to help the reader get familiar with the data that will be used for this project. Section 2 contains information with regards to dimensionality, cardinality, data types, etc.

In section 3, the focus will be on data celaning. This task includes type transformation, dropping features which are not used for prediction and more.

# 2. Getting familiar with the data

### 2.1 First impression

In [0]:
# Data set is loaded (patiences required, it is as "large" data set)
df_inspections = pd.read_csv("https://drive.switch.ch/index.php/s/nlURni4cnxItrbn/download")

In [6]:
# Sample row of the data set
df_inspections[df_inspections["Inspection ID"] == 2345616]

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
2,2345616,VIP FIT CLUB,VIP FIT CLUB,2446547.0,Restaurant,Risk 2 (Medium),3426 W DIVERSEY AVE,CHICAGO,IL,60647.0,11/14/2019,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.932069,-87.713294,"(-87.71329441237397, 41.932068626464286)"


In [7]:
# Shape of the data set
df_inspections.shape

(195979, 17)

As shown above, the data set cardinality is 195'979, while the dimensionality is 17.

### 2.2 Data types

In [8]:
df_inspections.dtypes

Inspection ID        int64
DBA Name            object
AKA Name            object
License #          float64
Facility Type       object
Risk                object
Address             object
City                object
State               object
Zip                float64
Inspection Date     object
Inspection Type     object
Results             object
Violations          object
Latitude           float64
Longitude          float64
Location            object
dtype: object

This table shows that the "Inspection Date" attribute is of type object. Below, it is converted to datetime. There are other issues, such as integer being interpreted as float numbers.

# 3. Data transformation

### 3.1 Type conversion

In [9]:
# Date conversion
df_inspections["Inspection Date"] =  pd.to_datetime(df_inspections["Inspection Date"], format = "%m/%d/%Y")
print("Date of first report : " + str(df_inspections.iloc[-1]["Inspection Date"].year))
print("Date of last report : " + str(df_inspections.iloc[0]["Inspection Date"].year))

Date of first report : 2010
Date of last report : 2019


### 3.2 Data reduction

Not all the features are important. Some features contain the same information ("Location", for instance, is equivalent to the combination of "Latitude" and "Longitude". Some columns are dropped to make the following step "lighter" and improve readability.

In [0]:
#Drop columns that will not be used
df_inspections = df_inspections.drop(["AKA Name", "City", "State", "Location"], axis = 1)

In [11]:
df_inspections.head(3)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
0,2345699,JET'S PIZZA,2163956.0,Restaurant,Risk 2 (Medium),2811 N ASHLAND AVE,60657.0,2019-11-15,Canvass,Pass w/ Conditions,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.932766,-87.668262
1,2345619,JAIPUR,2694084.0,Restaurant,Risk 1 (High),738 W RANDOLPH ST,60661.0,2019-11-14,Canvass,Pass,NaN,41.884518,-87.647304
2,2345616,VIP FIT CLUB,2446547.0,Restaurant,Risk 2 (Medium),3426 W DIVERSEY AVE,60647.0,2019-11-14,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.932069,-87.713294


Below, the table shows the possible outcomes of the inspection.

In [12]:
df_inspections["Results"].value_counts()

Pass                    105850
Fail                     37900
Pass w/ Conditions       27169
Out of Business          16866
No Entry                  6243
Not Ready                 1883
Business Not Located        68
Name: Results, dtype: int64

In [0]:
# In this project, only "Pass" and "Fail" will be considered
df_binary = df_inspections.query("Results == 'Fail' or Results == 'Pass'")

In [14]:
df_binary.shape

(143750, 13)

In [15]:
print(str(df_inspections.shape[0] - df_binary.shape[0]) + " rows were dropped.")

52229 rows were dropped.


### 3.3 Data cleaning

In [16]:
#Show how many NaN there are in each columns.
df_binary.isna().sum()

Inspection ID          0
DBA Name               0
License #             15
Facility Type        620
Risk                  34
Address                0
Zip                   28
Inspection Date        0
Inspection Type        1
Results                0
Violations         26994
Latitude             543
Longitude            543
dtype: int64

Below, rows for which there is a missing value with regards to "License #", "Zip", "Risk", "Inspection type", "Latitude", "Longitude" are dropped.

For violations the NaN are kept as is because the "Violations" fields are often empty for restaurants that passed the inspection.

In [17]:
# Drop NaN values
df_binary = df_binary.dropna(subset = ["License #", "Risk", "Zip", "Inspection Type", "Latitude", "Longitude"])
df_binary.isna().sum()

Inspection ID          0
DBA Name               0
License #              0
Facility Type        597
Risk                   0
Address                0
Zip                    0
Inspection Date        0
Inspection Type        0
Results                0
Violations         26808
Latitude               0
Longitude              0
dtype: int64

In [18]:
#How many rows were dropped ? (143750 - 143130) = 620
df_binary.shape

(143130, 13)

In [0]:
#Float to Int conversion when needed (could not be done before because of NaN values)
df_binary["License #"] = pd.to_numeric(df_binary["License #"]).astype(int)
df_binary["Zip"] = pd.to_numeric(df_binary["Zip"]).astype(int)

In [20]:
#Risk column cleaning
df_binary["Risk"].value_counts()

Risk 1 (High)      104358
Risk 2 (Medium)     27756
Risk 3 (Low)        11011
All                     5
Name: Risk, dtype: int64

The value "All" in the "Risk" column are dropped below.

In [0]:
df_binary = df_binary[df_binary["Risk"] != "All"]

As shown below, the "Facility Type" feature contains 450 different values, which will raise issues when encoded.

In [22]:
df_binary["Facility Type"].value_counts()

Restaurant                                         94627
Grocery Store                                      19085
School                                             10500
Children's Services Facility                        2444
Bakery                                              2183
                                                   ...  
MOBILE FROZEN DESSERTS DISPENSER-NON- MOTORIZED        1
PRODUCE VENDOR                                         1
KIDS CAFE                                              1
DISTRIBUTOR                                            1
PEDDLER                                                1
Name: Facility Type, Length: 450, dtype: int64

One way to reduce the number of facility types is to transform all the column into lower case.

In [0]:
df_binary["Facility Type"] = df_binary["Facility Type"].str.lower()

In [24]:
df_binary["Facility Type"].value_counts()

restaurant                        94627
grocery store                     19085
school                            10510
children's services facility       2444
bakery                             2183
                                  ...  
nutrition/herbalife                   1
gas station /subway mini mart.        1
grocery/liquor                        1
rooftop patio                         1
grocery/ restaurant                   1
Name: Facility Type, Length: 407, dtype: int64

There are still too many types. Here, they are selected based on their count value. The ones with the highest count are conserved such that the resulting set contains 97.5% of the df_binary dataset. The rest is dropped.

In [40]:
# List of the 100 first types in the previous table
list_types = list(df_binary["Facility Type"].value_counts().head(100).index)
myIndex = 0
# Start with empty DF witht the same attributes as df_binary
df_temp = df_binary[df_binary["Facility Type"] == ""]
# While under 97.5%, continue
while(df_temp.shape[0]/df_binary.shape[0] < 0.975 and myIndex <= len(list_types)) :
  # For each iteration, concatenate the temporary DF and the rows of the original DF 
  # which match the current facility type (defined by myIndex)
  df_temp = pd.concat([df_temp, df_binary[df_binary["Facility Type"] == list_types[myIndex]]], join="inner")
  myIndex += 1
# Prints the reached percentage
print("Reached : " + str(100*(df_temp.shape[0]/df_binary.shape[0])) + " %")

Reached : 97.52803493449782 %


Those are the final types that will be used.

In [41]:
df_temp["Facility Type"].value_counts()

restaurant                         94627
grocery store                      19085
school                             10510
children's services facility        2444
bakery                              2183
daycare (2 - 6 years)               2179
daycare above and under 2 years     1951
long term care                      1052
catering                             831
liquor                               720
mobile food dispenser                691
daycare combo 1586                   590
wholesale                            487
mobile food preparer                 457
golden diner                         456
hospital                             434
tavern                               251
daycare (under 2 years)              214
shared kitchen user (long term)      162
special event                        160
gas station                          103
Name: Facility Type, dtype: int64

The same procedure has to be used for the "Inspection type".

In [42]:
df_temp["Inspection Type"].value_counts()

canvass                              64733
license                              20919
canvass re-inspection                17448
complaint                            13343
license re-inspection                 7963
                                     ...  
license task 1474                        1
taskforce                                1
license daycare 1586                     1
task force liquor inspection 1474        1
sfp                                      1
Name: Inspection Type, Length: 85, dtype: int64

In [0]:
df_temp["Inspection Type"] = df_temp["Inspection Type"].str.lower()

In [63]:
# List of the 100 first types in the previous table
list_types = list(df_temp["Inspection Type"].value_counts().head(100).index)
myIndex = 0
# Start with empty DF witht the same attributes as df_temp
df_final = df_temp[df_temp["Inspection Type"] == ""]
# While under 99.7%, continue
while(df_final.shape[0]/df_temp.shape[0] < 0.997 and myIndex <= len(list_types)) :
  # For each iteration, concatenate the temporary DF and the rows of the original DF 
  # which match the current facility type (defined by myIndex)
  df_final = pd.concat([df_final, df_temp[df_temp["Inspection Type"] == list_types[myIndex]]], join="inner")
  myIndex += 1
# Prints the reached percentage
print("Reached : " + str(100*(df_final.shape[0]/df_temp.shape[0])) + " %")

Reached : 99.73779793247222 %


In [64]:
df_final["Inspection Type"].value_counts()

canvass                                   64733
license                                   20919
canvass re-inspection                     17448
complaint                                 13343
license re-inspection                      7963
complaint re-inspection                    6361
short form complaint                       5523
suspected food poisoning                    596
license-task force                          576
consultation                                496
tag removal                                 422
task force liquor 1475                      241
recent inspection                           194
suspected food poisoning re-inspection      170
complaint-fire                              137
short form fire-complaint                    99
Name: Inspection Type, dtype: int64

# 4. Export

The result is saved in a .csv file which can then be downloaded by opening the left pane and selecting "Files" on Google Colab.

In [0]:
df_final.to_csv("clean_food_inspections.csv", index=False)